In [16]:
import torch
from transformers import LongformerTokenizerFast

tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')

# preprocessing data

In [18]:
from datasets import load_dataset


dataset = load_dataset("natural_questions")

Using custom data configuration default
Reusing dataset natural_questions (/home/sri/.cache/huggingface/datasets/natural_questions/default/0.0.2/867dbbaf9137c1b83ecb19f5eb80559e1002ea26e702c6b919cfa81a17a8c531)


In [33]:
def convert_to_features(example, max_length=4096):
    try:
    #example = dataset['train'][0]
        start_idx, end_idx = example['annotations']['long_answer'][0]['start_token'], example['annotations']['long_answer'][0]['end_token']
        answer  = " ".join([itm for idx, itm in enumerate(example['document']['tokens']['token'][start_idx:end_idx]) if not example['document']['tokens']['is_html'][start_idx+idx]])
        context = " ".join([itm for idx, itm in enumerate(example['document']['tokens']['token'][:]) if not example['document']['tokens']['is_html'][idx]])
        query = example['question']['text']
        input_pairs =[query, context]
        #encodings = tokenizer.encode_plus(input_pairs, pad_to_max_length=True, truncation=True, max_length=max_length)
        encodings = tokenizer.encode_plus(input_pairs, padding='max_length', truncation=True, max_length=max_length)
        context_encodings = tokenizer.encode_plus(context)

        start_byte_idx = context.find(answer)
        end_byte_idx = start_byte_idx + len(answer)-1

        if start_idx==-1:
            #print(start_idx)
            encodings.update({'start_positions': 0,
                              'end_positions': 0,
                              'attention_mask': encodings['attention_mask']})
            
            for enc, val in encodings.items():
                if isinstance(enc, torch.Tensor):
                    encodings.update({
                        enc: val.to_list()
                    })
            return encodings
        start_positions_context = context_encodings.char_to_token(start_byte_idx)
        end_positions_context = context_encodings.char_to_token(end_byte_idx)

        sep_idx = encodings['input_ids'].index(tokenizer.sep_token_id)

        start_positions = start_positions_context + sep_idx + 1
        end_positions = end_positions_context + sep_idx + 1

        if end_positions > 4096:
              start_positions, end_positions = 0, 0

        encodings.update({'start_positions': start_positions,
                              'end_positions': end_positions,
                              'attention_mask': encodings['attention_mask']})
        for enc, val in encodings.items():
                if isinstance(enc, torch.Tensor):
                    encodings.update({
                        enc: val.to_list()
                    })
    except Exception as e:
        #print(start_idx, example)
        print(e, example['id'])
    return encodings

In [41]:
train_dataset = dataset['train'].select(indices=list(range(320)))
#train_dataset = dataset['train']
train_dataset = train_dataset.map(convert_to_features)
train_dataset.flatten_indices().save_to_disk("datasets/train_data_4096")

Loading cached processed dataset at /home/sri/.cache/huggingface/datasets/natural_questions/default/0.0.2/867dbbaf9137c1b83ecb19f5eb80559e1002ea26e702c6b919cfa81a17a8c531/cache-8f68a18e877fb425.arrow
Loading cached processed dataset at /home/sri/.cache/huggingface/datasets/natural_questions/default/0.0.2/867dbbaf9137c1b83ecb19f5eb80559e1002ea26e702c6b919cfa81a17a8c531/cache-233a19e628ecf940.arrow


In [42]:
valid_dataset = dataset['validation'].select(indices=list(range(320)))
#valid_dataset = dataset['validation']
valid_dataset = valid_dataset.map(convert_to_features, load_from_cache_file=False)

train_dataset.flatten_indices().save_to_disk("datasets/validation_data_4096")

Loading cached processed dataset at /home/sri/.cache/huggingface/datasets/natural_questions/default/0.0.2/867dbbaf9137c1b83ecb19f5eb80559e1002ea26e702c6b919cfa81a17a8c531/cache-233a19e628ecf940.arrow


In [39]:
# # loading data

# import datasets

# datasets.load_from_disk("dataset/train_data_4096/")